In [315]:
using PyPlot
import kin_init
import MD_calc

reload("MD_calc")

In [306]:
Ni = 2#8; #number of atoms per side in original cubic configuration 
Ts = 1.0; #desired (and initial) temperature in LJ units (Ts = kB*T/epsilon)
ns = 0.05; #LJ number density ns = n*sigma^3
epsilon = 1.65e-21; #LJ energy [J]
sigma = 3.4e-10; #LJ diameter [m]
m = 39.948*1.660538921e-27; #molecule mass [kg]
kB = 1.38065e-23; #Boltzmann constant [J/K]

1.38065e-23

In [324]:
rc = 3; #cut-off radius, in LJ units
STEPS = 1000#1e4; #number of timesteps
STEPS_thermostat = round(STEPS/10) #3*10^3; #number of steps to leave thermostat on until
STEPS_equilib = round(STEPS/2) #4*10^3; #number of steps before starting to average
alpha = 0.1; #thermostat relaxation parameter 
dt = 0.005; #time step, in LJ units

0.005

In [308]:
N = Ni^3; #number of atoms 
Vs = N/ns; #volume of domain, in LJ units, Vs = V/sigma^3
Ls = Vs^(1/3); #length of the domain in each direction, in LJ units
T_set = Ts*epsilon/kB; #set temperature, in Kelvin
P_LRC = 32/9*pi*ns^2*rc^-9. - 16/3*pi*ns^2*rc^-3.; #long-range P correction
E_LRC = 8/9*pi*ns*rc^-9. - 8/3*pi*ns*rc^-3.; #long-range U correction (per particle)
rc2 = rc^2; #square of cut-off radius

9

In [325]:
r = zeros(N,3); 
v = zeros(N,3);  
F = zeros(N,3); 
U = zeros(N,1); 
Res = zeros(STEPS,2); 
positions = zeros(N,3,STEPS);
#forces=zeros(N,N,3,STEPS);

r,v=kin_init.initialize(Ls,Ni,Ts);

In [326]:
c = [sum(v[:,1]) sum(v[:,2]) sum(v[:,3])]/N;
v = broadcast(-,v,c);

In [327]:
for t = 1:1:STEPS   

    Uij = zeros(N,N);
    Fij = zeros(N,N,3);
    
    (Fij,Uij,rij)= MD_calc.force_calculation_mod(N,r,Ls,rc2);
    
       
    F[:,1] = sum(Fij[:,:,1]')';
    F[:,2] = sum(Fij[:,:,2]')';
    F[:,3] = sum(Fij[:,:,3]')';

    (rnew,vnew,T_inst)= MD_calc.take_one_step_good(N,r,v,F,t,dt,Ls,STEPS_thermostat,alpha,Ts);
    
    r = rnew;
    v = vnew;
    
    #[P,U]= P_and_U(N,Vs,rij,Fij,Uij,T_inst,P_LRC,E_LRC);
    
    Res[t,:] = [t T_inst]; #P U];

    if mod(t,10)==0
        print([t T_inst],"\n") # write to screen        
    end
    
end

[10.0 0.6994940426474635]
[20.0 0.8952200515495603]
[30.0 0.9634654910205422]
[40.0 0.987261204399223]
[50.0 0.9955582566211687]
[60.0 0.9984512598473441]
[70.0 0.9994599876994519]
[80.0 0.9998117093534097]
[90.0 0.9999343471110615]
[100.0 0.9999771082530972]
[110.0 0.9999771082530972]
[120.0 0.9999771082530972]
[130.0 0.9999771082530972]
[140.0 0.9999771082530972]
[150.0 0.9999771082530972]
[160.0 0.9999771082530972]
[170.0 0.9999771082530972]
[180.0 0.9999771082530972]
[190.0 0.9999771082530972]
[200.0 0.9999771082530972]
[210.0 0.9999771082530972]
[220.0 0.9999771082530972]
[230.0 0.9999771082530972]
[240.0 0.9999771082530972]
[250.0 0.9999771082530972]
[260.0 0.9999771082530972]
[270.0 0.9999771082530972]
[280.0 0.9999771082530972]
[290.0 0.9999771082530972]
[300.0 0.9999771082530972]
[310.0 0.9999771082530972]
[320.0 0.9999771082530972]
[330.0 0.9999771082530972]
[340.0 0.9999771082530972]
[350.0 0.9999771082530972]
[360.0 0.9999771082530972]
[370.0 0.9999771082530972]
[380.0 0.99

In [272]:
#INITIAL CALCULATION OF FORCES AND ENERGIES # NOT IN USE
(F,U) = MD_calc.force_calculation_mod(N,r,Ls,rc2); 

tic()
for t = 1:STEPS
    (rnew,vnew) = MD_calc.take_half_step(r,v,F,dt,Ls)
    (F,U) = MD_calc.force_calculation_mod(N,rnew,Ls,rc2);
    (vnew,T_inst) = MD_calc.take_one_step(N,vnew,F,t,dt,STEPS_thermostat,alpha,Ts);   

    r = rnew;
    v = vnew;

    #Axes3D.scatter(rnew[:,1],rnew[:,2],rnew[:,3]);
    #if(abs(max(F))>1.e12)
    #    disp('Gone! Everything is lost!');
    #    break;
    #end

    #[P,u]= P_and_U_modified(N,Vs,rnew,F,U,T_inst,P_LRC,E_LRC);
    
    Res[t,:] = [t T_inst]; #P u];
    positions[:,:,t]=rnew;
    #forces[:,:,:,t]=F;

    if mod(t,1000)==0
        print([t T_inst],"\n") # write to screen        
    end
end
toc()

[1000.0 5.6279778467098165]
[2000.0 24.47208617366038]
[3000.0 76.45556017876133]
[4000.0 58.98029577005315]
[5000.0 21.62774809636622]
[6000.0 15679.852031853548]
[7000.0 206215.88917284465]
[8000.0 1.6356144809370922e7]
[9000.0 1.6672445971795943e7]
[10000.0 1.6697952116989473e7]
[11000.0 1.6638771120652635e7]
[12000.0 3.158194312231987e14]
[13000.0 3.1581960299114256e14]
[14000.0 3.1581969350781406e14]
[15000.0 3.158155720224756e14]
[16000.0 3.158156798005525e14]
[17000.0 3.158156798005525e14]
[18000.0 3.158156798005525e14]
[19000.0 3.158156798005525e14]
[20000.0 3.158156798005525e14]
[21000.0 3.158156798005525e14]
[22000.0 3.158156798005525e14]
[23000.0 3.158156798005525e14]
[24000.0 3.158156798005525e14]
[25000.0 3.158156798005525e14]
[26000.0 3.158156798005525e14]
[27000.0 3.158156798005525e14]
[28000.0 3.158156798005525e14]
[29000.0 3.158156798005525e14]
[30000.0 3.158156798005525e14]
[31000.0 3.158156798005525e14]
[32000.0 3.158156798005525e14]
[33000.0 3.158156798005525e14]
[3

9.924056809

In [273]:
Res

50000x2 Array{Float64,2}:
     1.0  0.265421  
     2.0  0.33888   
     3.0  0.404992  
     4.0  0.464494  
     5.0  0.518045  
     6.0  0.566241  
     7.0  0.609618  
     8.0  0.648657  
     9.0  0.683792  
    10.0  0.715414  
    11.0  0.743874  
    12.0  0.769487  
    13.0  0.79254   
     ⋮              
 49989.0  3.15816e14
 49990.0  3.15816e14
 49991.0  3.15816e14
 49992.0  3.15816e14
 49993.0  3.15816e14
 49994.0  3.15816e14
 49995.0  3.15816e14
 49996.0  3.15816e14
 49997.0  3.15816e14
 49998.0  3.15816e14
 49999.0  3.15816e14
 50000.0  3.15816e14

In [303]:
positions[:,:,30000]

8x3 Array{Float64,2}:
       1.31886    3.98007     -1061.79    
       2.69932    3.42899         3.01493 
    1064.53       0.0252382       0.449126
       2.39859    1.15264    147597.0     
      -8.66334e9  0.779147        3.87328 
      -2.29203e6  3.82711         0.788396
    7939.03       4.6296          3.34378 
 -252561.0        2.59753         5.30376 

In [275]:
Ls

5.428835233189813

In [60]:
vnew

8x3 Array{Float64,2}:
 -0.73974   -0.532541   -0.322447
  0.683656  -0.32748    -0.579109
 -0.805619   0.375255    0.183346
  0.333403   0.265038    1.16687 
  0.298865   0.385898   -0.130192
  0.460354   0.0169396  -0.790359
 -0.498688  -0.338197    0.243944
  0.267706   0.155025    0.228008

In [61]:
sum(vnew[:,1])

-6.28058133331777e-5

In [62]:
[sum(vnew[:,1]) sum(vnew[:,2]) sum(vnew[:,3])]/N

1x3 Array{Float64,2}:
 -7.85073e-6  -7.83496e-6  7.81809e-6

In [86]:
Res

5x2 Array{Float64,2}:
 1.0  0.315235
 2.0  0.315235
 3.0  0.315235
 4.0  0.315235
 5.0  0.315235

In [76]:
(vnew,T_inst) = MD_calc.take_one_step(N,v,F,1,dt,STEPS_thermostat,alpha,Ts);   



In [77]:
vnew

8x3 Array{Float64,2}:
   0.823558  NaN  NaN
 NaN         NaN  NaN
 NaN         NaN  NaN
 NaN         NaN  NaN
 NaN         NaN  NaN
 NaN         NaN  NaN
 NaN         NaN  NaN
 NaN         NaN  NaN

In [78]:
T_inst

NaN

In [75]:
v

8x3 Array{Float64,2}:
 0.823558   0.35244   0.839717
 1.31097    0.296109  0.943219
 0.106339   0.170874  0.113927
 0.998036   0.845631  1.24674 
 0.0384565  1.83876   1.49377 
 0.813857   0.125209  0.571669
 1.52412    1.80222   0.170394
 1.26844    0.631742  1.50023 

In [79]:
F

8x8x3 Array{Float64,3}:
[:, :, 1] =
   0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN    NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN    NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN    NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN    NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN    NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN    NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN    NaN  NaN  NaN  NaN  NaN  NaN  NaN

[:, :, 2] =
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN

[:, :, 3] =
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 N

In [80]:
    (F,U) = MD_calc.force_calculation_mod(N,r,Ls,rc2);



In [81]:
F

8x8x3 Array{Float64,3}:
[:, :, 1] =
  0.0          -0.0           0.0219937    …   0.00138065    0.000272841
 -0.0           0.0           0.00138065       0.000272841   0.00138065 
 -0.0219937    -0.00138065    0.0             -0.0          -0.0        
 -0.00138065   -0.0219937    -0.0             -0.0          -0.0        
 -0.0          -0.0           0.00138065       0.0219937     0.00138065 
 -0.0          -0.0           0.000272841  …   0.00138065    0.0219937  
 -0.00138065   -0.000272841  -0.0              0.0          -0.0        
 -0.000272841  -0.00138065   -0.0             -0.0           0.0        

[:, :, 2] =
  0.0           0.0219937    -0.0          …  -0.0           0.000272841
 -0.0219937     0.0          -0.00138065      -0.000272841  -0.0        
 -0.0           0.00138065    0.0             -0.0           0.00138065 
 -0.00138065   -0.0          -0.0219937       -0.00138065   -0.0        
 -0.0           0.00138065   -0.0             -0.0           0.00138065 
 -

In [82]:
(vnew,T_inst) = MD_calc.take_one_step(N,v,F,1,dt,STEPS_thermostat,alpha,Ts);   



In [83]:
vnew

8x3 Array{Float64,2}:
 0.823558   0.35244   0.839772
 1.31097    0.296109  0.943222
 0.106284   0.17087   0.113927
 0.998033   0.845576  1.24674 
 0.0384565  1.83876   1.49377 
 0.813857   0.125209  0.57167 
 1.52412    1.80222   0.170394
 1.26844    0.631739  1.50023 

In [84]:
T_inst

0.3152323559320227

In [160]:
v

8x3 Array{Float64,2}:
  0.411043  -0.0468727   0.625911 
 -0.300794   0.362473   -0.794991 
  0.363557  -0.458597    0.143336 
 -0.289557   0.973574    0.368802 
 -0.143763   0.303684   -0.556069 
  0.544176  -0.775059    0.806894 
 -0.254854   0.30907    -0.621846 
 -0.329807  -0.668271    0.0279619

In [173]:
vx = v[:,1];
    vy = v[:,2];
    vz  = v[:,3];

    ax = F[:,1];
    ay = F[:,2];
    az = F[:,3];

    vxnew = vx + 0.5*dt*ax;
    vynew = vy + 0.5*dt*ay;
    vznew = vz + 0.5*dt*az;

    vnew = hcat(vxnew, vynew, vznew);

    c = [sum(vnew[:,1]) sum(vnew[:,2]) sum(vnew[:,3])]/N;
    v_for_temp = broadcast(-,vnew,c);

    T_inst = 1./(3.*N)*sum(v_for_temp.^2);


In [174]:
T_inst

0.2508661454437064

In [175]:
        v_scal = sqrt(1 + 1*(1/T_inst - 1.));


In [176]:
v_scal

1.9965443948073118

In [177]:
        vnew = v_scal*vnew;


In [178]:
vnew

8x3 Array{Float64,2}:
  0.820665  -0.0935835   1.24977  
 -0.600549   0.723693   -1.58723  
  0.725748  -0.915615    0.286178 
 -0.578121   1.94367     0.73633  
 -0.287029   0.606318   -1.11021  
  1.08647   -1.54744     1.611    
 -0.508834   0.617071   -1.24154  
 -0.658476  -1.33424     0.0558272

In [180]:
T_inst = 1./(3.*N)*sum(vnew.^2);


In [181]:
T_inst

1.0000000002442955

In [223]:
xnew=[1 2 3 4 5 6 3 1 -10]

1x9 Array{Int64,2}:
 1  2  3  4  5  6  3  1  -10

In [ ]:
Ls=